# Smart Aliens:
## Using Statistical Interference to Find Exoplanets that Could Harbor Intelligent Alien Life
---

### by Julian Estrada & William Walker

- 
 

In [11]:
#### IMPORT Everything here ####
import ast
import bs4
import flask
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import requests
import scipy
import scipy.stats as st
import sqlalchemy




from bs4 import BeautifulSoup

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float

#from selenium import webdriver
#from splinter import Browser

# Part I: Using Flask

In [ ]:
## API Flask ##

## Import via loading in csv file until API works

 

In [12]:
## Read in csv file ##
exoplanet_metadata_path = "data/exoplanet-archive.csv"
exoplanet_metadata = pd.read_csv(exoplanet_metadata_path)


/var/folders/hf/mp5v5rls3hg252gfcnygxxpw0000gn/T/ipykernel_88366/986843285.py:3: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  exoplanet_metadata = pd.read_csv(exoplanet_metadata_path)


# Part II: Cleaning the Data

In [67]:
## Create exoplanet dataframe with columns we want ##
exoplanet_df = exoplanet_metadata[[
    "pl_name", "hostname", "disc_year", "discoverymethod", "pl_orbper", "pl_orbsmax",\
          "pl_rade", "pl_masse", "pl_bmasse", "pl_insol", "pl_eqt", "st_spectype", "st_teff",\
              "glat", "glon", "sy_dist"
]]


In [69]:
## Shortcut of Math Items ##
pi = np.pi

sin = np.sin
cos = np.cos
arcsin = np.arcsin
arccos = np.arccos
sqrt = np.sqrt

In [71]:

## Convert distance to Light-Years ##
lightyear = ((2 ** 2) * 3 * (5 ** 3) * 877 * 7789 ) / (pi * 7 * 487 * 293339)
exoplanet_df["sy_dist"] *= lightyear

## Experimental average gravity of Earth in m/s^2 ##
g_earth = 9.8203

## Insert Gravity columns ##
exoplanet_df["pl_gravitye"] = exoplanet_df["pl_masse"] / (exoplanet_df["pl_rade"] ** 2)
exoplanet_df["pl_gravity"] = round(g_earth * exoplanet_df["pl_gravitye"], 4)

## X and Y value coordinates in Light-Years ##
exoplanet_df["xVal"] = exoplanet_df["sy_dist"] * -sin( exoplanet_df["glon"] * pi / 180 )\
      * cos( exoplanet_df["glat"] * pi / 180 )
exoplanet_df["yVal"] = exoplanet_df["sy_dist"] * cos( exoplanet_df["glon"] * pi / 180 )\
      * cos( exoplanet_df["glat"] * pi / 180 )



In [72]:
## Reorder Columns ##
exoplanet_df = exoplanet_df[[
    "pl_name", "hostname", "disc_year", "discoverymethod", "pl_orbper", "pl_orbsmax",\
          "pl_rade", "pl_masse", "pl_bmasse", "pl_gravitye", "pl_gravity", "pl_insol", "pl_eqt",\
              "st_spectype", "st_teff", "glat", "glon", "sy_dist", "xVal", "yVal"
]]


exoplanet_df.head(25)


,pl_name,hostname,disc_year,discoverymethod,pl_orbper,pl_orbsmax,pl_rade,pl_masse,pl_bmasse,pl_gravitye,pl_gravity,pl_insol,pl_eqt,st_spectype,st_teff,glat,glon,sy_dist,xVal,yVal
0,11 Com b,11 Com,2007,Radial Velocity,NaN,1.210,NaN,NaN,5434.70000,NaN,NaN,NaN,NaN,NaN,NaN,78.28058,264.13775,991.278977,200.294794,-20.565087
1,11 Com b,11 Com,2007,Radial Velocity,326.03000,1.290,NaN,NaN,6165.60000,NaN,NaN,NaN,NaN,G8 III,4742.00,78.28058,264.13775,991.278977,200.294794,-20.565087
2,11 Com b,11 Com,2007,Radial Velocity,323.21000,1.178,NaN,NaN,4914.89849,NaN,NaN,NaN,NaN,G8 III,4874.00,78.28058,264.13775,991.278977,200.294794,-20.565087
3,11 UMi b,11 UMi,2009,Radial Velocity,516.21997,1.530,NaN,NaN,4684.81420,NaN,NaN,NaN,NaN,NaN,4213.00,41.04437,108.71900,1333.139517,-952.270886,-322.677916
4,11 UMi b,11 UMi,2009,Radial Velocity,NaN,1.510,NaN,NaN,3432.40000,NaN,NaN,NaN,NaN,NaN,NaN,41.04437,108.71900,1333.139517,-952.270886,-322.677916
5,11 UMi b,11 UMi,2009,Radial Velocity,516.22000,1.540,NaN,NaN,3337.07000,NaN,NaN,NaN,NaN,K4 III,4340.00,41.04437,108.71900,1333.139517,-952.270886,-322.677916
6,14 And b,14 And,2008,Radial Velocity,186.76000,0.775,NaN,NaN,1131.15130,NaN,NaN,NaN,NaN,K0 III,4888.00,-21.05141,106.41269,802.506991,-718.427961,-211.617761
7,14 And b,14 And,2008,Radial Velocity,NaN,0.680,NaN,NaN,1017.00000,NaN,NaN,NaN,NaN,NaN,NaN,-21.05141,106.41269,802.506991,-718.427961,-211.617761
8,14 And b,14 And,2008,Radial Velocity,185.84000,0.830,NaN,NaN,1525.50000,NaN,NaN,NaN,NaN,K0 III,4813.00,-21.05141,106.41269,802.506991,-718.427961,-211.617761
9,14 Her b,14 Her,2002,Radial Velocity,1773.40000,2.770,NaN,NaN,1474.67000,NaN,NaN,NaN,NaN,K0 V,NaN,46.94447,69.16849,190.760190,-121.719971,46.313695


# Part III: Further manipulation

# Part IV: Put forth into javascript

 